<a href="https://colab.research.google.com/github/Abhiroop25902/Video-AI-Upscaling/blob/main/Video_AI_Upscaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup 

Just Run These Code Once to download and import all the pre-requsites

I have already built OpenCV from Source for GPU support, and the first line downloads it's python source code

Second Line downloads the EDSR Model

In [1]:
!gdown --id 1--R75ERdRPjwh2XHWiRHno1FIuuA3iVg --output cv2.cpython-37m-x86_64-linux-gnu.so #enable GPU support
!gdown --id 11wBtbU0smZKV2UG49e9RzyKkiIAYAXHn --output EDSR_x3.pb 

Downloading...
From: https://drive.google.com/uc?id=1--R75ERdRPjwh2XHWiRHno1FIuuA3iVg
To: /content/cv2.cpython-37m-x86_64-linux-gnu.so
1.01GB [00:05, 190MB/s]
Downloading...
From: https://drive.google.com/uc?id=11wBtbU0smZKV2UG49e9RzyKkiIAYAXHn
To: /content/EDSR_x3.pb
38.5MB [00:00, 83.1MB/s]


Upgrading OpenCV and OpenCV contrib, Colab default doesn't have `dnn_superres` as of June 2021

In [2]:
!pip install --upgrade opencv-python
!pip install --upgrade opencv-contrib-python

     |████████████████████████████████| 51.0MB 54kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 57.4MB 53kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


Importing Required Stuff

In [3]:
import cv2 as cv
from cv2 import dnn_superres
from tqdm.notebook import tqdm #for google colab progressbar
from time import time # for testing purposes

# Import Your Own Video




You can directly drag and drop your video to the file explorer in the google colab, but I would not suggest it, primarily because Colab shows only a circular progress bar and no "time remaining" thing.

Here's what I do:
-	Upload your video to google drive
-   Make it's visibility to "anyone with the link can view"
-   It's share link link will be something like 
    -   https ://drive.google.com/file/d/`<some key>`/view?usp=sharing
-   Add command `!gdown --id <the key> --output <file name>` to any code snippet

I have done this already for a sample `network.mp4` file 

NOTE: RAM Requirements are heavily dependant on original width and height of Video, the max resolution this was able to process in Colab Free was 640x360, anything more than that requires Colab Pro

Additional Context: 
-   900 frames  640x360 video should take 37:44 min to process (2.95 sec/frame)
-   120 frames  500x280 video should take 03:48 min to process (1.89 sec/frame)



In [4]:
# for network.mp4
# https://drive.google.com/file/d/18oenWel4G3W2DXMZoKLrwZl3K5yUG__f/view?usp=sharing
!gdown --id 18oenWel4G3W2DXMZoKLrwZl3K5yUG__f --output network.mp4

# for earth.mp4
# https://drive.google.com/file/d/1nrrDVyQBbBo9Gol8T-E-09bnhi4dJ4GS/view?usp=sharing
# !gdown --id 1nrrDVyQBbBo9Gol8T-E-09bnhi4dJ4GS --output earth.mp4

Downloading...
From: https://drive.google.com/uc?id=18oenWel4G3W2DXMZoKLrwZl3K5yUG__f
To: /content/network.mp4
100% 169k/169k [00:00<00:00, 5.35MB/s]


put `path` as your `<file name>`

In [5]:
path = "network.mp4"
# path = "earth.mp4"

# Inside Functions

These two functions are main processing blocks 

This function loop through the video once and returns fps, total_frames, width and height of the video

In [6]:
def video_property(filepath):
    "return (fps, frames, width, height)"

    cap = cv.VideoCapture(filepath)

    if (not cap.isOpened()):
        raise ValueError('File Not Found: Check given path')

    fps = cap.get(cv.CAP_PROP_FPS)
    frames = 0
    height = width = 0

    while True:
        ret, img = cap.read()

        if not ret:
            break

        if(frames == 0):
            height = img.shape[0]
            width = img.shape[1]

        frames = frames + 1

    cap.release()

    return fps, frames, width, height

This is the main processing function, this imports the video and outputs 3 time upscaled video

In [7]:
def generate_output_video(filepath):

    #these parameters depends on video and should be changed
    video_fps, video_total_frame, input_video_width, input_video_height = video_property(filepath)

    #multiplied by 3 as we are incresing output size by a factor of 3
    output_video_width = 3*input_video_width 
    output_video_height = 3*input_video_height

    cap = cv.VideoCapture(filepath)

    # Define the codec and create VideoWriter object
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    out = cv.VideoWriter('output.mp4',fourcc, video_fps,(output_video_width,output_video_height))

    
    # Create an Super-Res object
    sr = dnn_superres.DnnSuperResImpl_create()

    # Read the desired model
    sr.readModel("EDSR_x3.pb")
    
    # Set the desired model and scale to get correct pre- and post-processing
    sr.setModel("edsr", 3)

    #setting dnn to use CUDA
    sr.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
    sr.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA)

    # initial_time=time()
    for i in tqdm (range (video_total_frame), desc="Processing..."):    
        # Read image
        ret,image = cap.read()

        if not ret:
            break

        # Upscale the image
        result = sr.upsample(image)

        out.write(result)


    #release capture and output videos
    cap.release()
    out.release()
    print("Processing Completed, Download 'output.mp4' to View Results")
    # print(f"Time taken to process the input: {time()-initial_time} seconds")

# Main Executing Code

This will generate `output.mp4` upscaled to 3 times height and width of original video

In [8]:
generate_output_video(path)


Processing Completed, Download 'output.mp4' to View Results
